In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
path = "/content/drive/MyDrive/TR PDDN"
os.chdir(path)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import numpy as np


#Hasilkan Dataset
image_size = (180, 180)
batch_size = 8

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "Training",
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "Validation",
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
#Data Augmentation
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
    ]
)
#Setting Performa
train_ds = train_ds.prefetch(buffer_size=32)
val_ds = val_ds.prefetch(buffer_size=32)
#Buat Model
def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    # Image augmentation block
    x = data_augmentation(inputs)

    # Entry block
    x = layers.Rescaling(1.0 / 255)(x)
    x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [128, 256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)


model = make_model(input_shape=image_size + (3,), num_classes=2)
keras.utils.plot_model(model, show_shapes=True)
epochs = 5

callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"),
]
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)
model.fit(
    train_ds, epochs=epochs, callbacks=callbacks, validation_data=val_ds,

)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 47009 files belonging to 2 classes.
Using 37608 files for training.
Found 11649 files belonging to 2 classes.
Using 2329 files for validation.
Epoch 1/5
4701/4701 [==============================] - 8416s 2s/step - loss: 0.3019 - accuracy: 0.8739 - val_loss: 0.1720 - val_accuracy: 0.9330
Epoch 2/5
4701/4701 [==============================] - 342s 73ms/step - loss: 0.2030 - accuracy: 0.9245 - val_loss: 0.1497 - val_accuracy: 0.9511
Epoch 3/5
4701/4701 [==============================] - 345s 73ms/step - loss: 0.1687 - accuracy: 0.9390 - val_loss: 0.1298 - val_accuracy: 0.9571
Epoch 4/5
4701/4701 [==============================] - 344s 73ms/step - loss: 0.1546 - accuracy: 0.9459 - val_loss: 0.1070 - val_accuracy: 0.9575
Epoch 5/5
4701/4701 [==============================] - 344s 73ms/step - loss: 0.1408 - accuracy: 0.9490 - val_loss: 0.1108 - val_accuracy: 

In [ ]:
#Inference
img = keras.preprocessing.image.load_img(
    "/content/drive/MyDrive/TR PDDN/Validation/female/113099.jpg.jpg", target_size=image_size
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create batch axis

predictions = model.predict(img_array)
score = predictions[0]
print(
    "This image is %.2f percent female and %.2f percent male."
    % (100 * (1 - score), 100 * score)
)

This image is 100.00 percent female and 0.00 percent male.


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Jul  6 11:32:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    32W /  70W |   7700MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import numpy as np
#Latih dan temukan bobot untuk eksperimen sebelumnya
#Jalankan prediction dan tangkap hasil prediksinya
Y_pred = model.fit(val_ds)
y_pred = np.argmax(Y_pred ,axis =1)
print('Confusion Matrix')

#mengambil data label
true_categories = tf.concat([y for x, y in val_ds], axis=0)

#mencari confusion matrix berdasarkan API dari sklearn.
confusion_matrix = confusion_matrix(true_categories, y_pred)
print(confusion_matrix)
print('Classification Report')
print(classification_report(true_categories, y_pred))


NameError: ignored